# SCREEN CAPTURE

In [2]:
## packages for Screen capture 
import pyscreenshot as ImageGrab
import time
## packages for generating dataset
import pandas as pd
import numpy as np
import cv2
import csv
import glob
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
## package for saving model
import joblib

In [32]:
def capture_image():
    images_folder='captured_images/9/'
    for i in range(0,50):
        time.sleep(8)
        im=ImageGrab.grab(bbox=(60,250,550,700)) #x1,y1,x2,y2
        print("saved.....",i)
        im.save(images_folder+str(i)+'.png')
        print("clear screen and re draw again........")

saved..... 0
clear screen and re draw again........
saved..... 1
clear screen and re draw again........
saved..... 2
clear screen and re draw again........
saved..... 3
clear screen and re draw again........
saved..... 4
clear screen and re draw again........
saved..... 5
clear screen and re draw again........
saved..... 6
clear screen and re draw again........
saved..... 7
clear screen and re draw again........
saved..... 8
clear screen and re draw again........
saved..... 9
clear screen and re draw again........
saved..... 10
clear screen and re draw again........
saved..... 11
clear screen and re draw again........
saved..... 12
clear screen and re draw again........
saved..... 13
clear screen and re draw again........
saved..... 14
clear screen and re draw again........
saved..... 15
clear screen and re draw again........
saved..... 16
clear screen and re draw again........
saved..... 17
clear screen and re draw again........
saved..... 18
clear screen and re draw again........
sav

# Generating Dataset

In [38]:
header=["label"]
for i in range(0,784):
    header.append("pixel"+str(i))
with open('digit_dataset.csv','a') as f:
    writer= csv.writer(f)
    writer.writerow(header)
for label in range(10):
    dirList = glob.glob("captured_images/"+str(label)+"/*.png")
    
    for img_path in dirList:
        im=cv2.imread(img_path)
        im_gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        im_gray=cv2.GaussianBlur(im_gray,(15,15),0)
        roi=cv2.resize(im_gray,(28,28),interpolation=cv2.INTER_AREA)
        
        data=[]
        data.append(label)
        rows,cols=roi.shape
        
        for i in range(rows):
            for j in range(cols):
                k=roi[i,j]
                if k>100:
                    k=1
                else:
                    k=0
                data.append(k)
        with open('digit_dataset.csv','a') as f:
            writer= csv.writer(f)
            writer.writerow(data)

In [40]:
data = pd.read_csv("digit_dataset.csv")
data=shuffle(data)
data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
159,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
183,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
366,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
277,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
444,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
459,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
435,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
X1=data.values[:,1:]
y1=data.values[:,0]

In [164]:
X1_train,X1_test,y1_train,y1_test = train_test_split(X1,y1,test_size=0.2,random_state=42)

In [165]:
print(X1_train.shape,X1_test.shape,y1_train.shape,y1_test.shape)

(400, 28, 28) (100, 28, 28) (400,) (100,)


In [186]:
X1_train = X1_train.reshape(X1_train.shape[0], 28, 28, 1).astype('float32')
X1_test = X1_test.reshape(X1_test.shape[0], 28, 28, 1).astype('float32')
y1_train = np_utils.to_categorical(y1_train)
y1_test = np_utils.to_categorical(y1_test)

In [187]:
print(X1_train.shape,X1_test.shape,y1_train.shape,y1_test.shape)

(400, 28, 28, 1) (100, 28, 28, 1) (400, 10) (100, 10)


# CNN MODEL

In [269]:
model_cnn=keras.Sequential([    
    #cnn
    keras.layers.Conv2D(filters=32,kernel_size=(2,2),activation='relu',input_shape=(28,28,1)),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(filters=42,kernel_size=(2,2),activation='relu',input_shape=(28,28,1)),
    keras.layers.MaxPool2D((2,2)),
    # dense
    keras.layers.Flatten(),
    keras.layers.Dense(3000,activation='relu'),
    keras.layers.Dense(1000,activation='relu'),
    keras.layers.Dense(10,activation='softmax')
])
model_cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_cnn.fit(X1_train,y1_train,epochs=10)

Epoch 1/10
13/13 [==============================] - 2s 97ms/step - loss: 1.6136 - accuracy: 0.4400
Epoch 2/10
13/13 [==============================] - 1s 89ms/step - loss: 0.5953 - accuracy: 0.8050
Epoch 3/10
13/13 [==============================] - 1s 89ms/step - loss: 0.2398 - accuracy: 0.9200
Epoch 4/10
13/13 [==============================] - 1s 89ms/step - loss: 0.1056 - accuracy: 0.9625
Epoch 5/10
13/13 [==============================] - 1s 87ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 6/10
13/13 [==============================] - 1s 86ms/step - loss: 0.0082 - accuracy: 0.9975
Epoch 7/10
13/13 [==============================] - 1s 85ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 8/10
13/13 [==============================] - 1s 86ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 9/10
13/13 [==============================] - 2s 119ms/step - loss: 6.0917e-04 - accuracy: 1.0000
Epoch 10/10
13/13 [==============================] - 2s 130ms/step - loss: 4.5736e-04 - accuracy: 1.0000

In [271]:
joblib.dump(model_cnn,"model/digit_rec_cnn")

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-04 22:31:34         3570
metad

['model/digit_rec_cnn']

In [270]:
model_cnn.evaluate(X1_test,y1_test)

4/4 [==============================] - 0s 15ms/step - loss: 0.0546 - accuracy: 0.9800


[0.05462278425693512, 0.9800000190734863]

# FINAL PREDICTION MODEL

In [3]:
final_model=joblib.load("model/digit_rec_cnn")
image_folder="img/"
while True:
    img=ImageGrab.grab(bbox=(60,200,550,750)) #x1,y1,x2,y2
    
    img.save(image_folder+"img.png")
    im=cv2.imread(image_folder+"img.png")
    im_gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    im_gray=cv2.GaussianBlur(im_gray,(15,15),0)
    
    ret,im_th=cv2.threshold(im_gray,100,255,cv2.THRESH_BINARY)
    roi=cv2.resize(im_th,(28,28),interpolation=cv2.INTER_AREA)
    
    rows,cols=roi.shape
    
    X_pred=[]
    for i in range(rows):
        for j in range(cols):
            k=roi[i,j]
            if k>100:
                k=1
            else:
                k=0
            X_pred.append(k)
    X_pred=np.array(X_pred)
    X_pred=X_pred.reshape(1,28,28,1)
    predictions=final_model.predict(X_pred)
    print("Prediction:",predictions.argmax())
    cv2.putText(im,"MODEL PREDICTION IS:"+str(predictions.argmax()),(20,20),0,0.8,(0,255,0),2,cv2.LINE_AA)
    
    cv2.startWindowThread()
    cv2.namedWindow("Result")
    cv2.imshow("Result",im)
    cv2.waitKey(10000)
    if cv2.waitKey(1)==13:
        break
cv2.destroyAllWindows()

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-04 22:31:34         3570
metadata.json                                  2023-03-04 22:31:34           64
variables.h5                                   2023-03-04 22:31:34     90704120
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...metrics\mean
......vars
.........0
.........1
...metrics\mean_metric_wrapper
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
